In [1]:
import imaplib
import clipboard
import email
import pandas as pd
import pyautogui
import re
import time
import smtplib
import yaml
from email.mime.text import MIMEText

In [9]:
with open("credentials.yml") as f:
    content = f.read()
    
# from credentials.yml import user name and password
my_credentials = yaml.load(content, Loader=yaml.FullLoader)


### data extraction from gmail

In [15]:
ORG_EMAIL = "@gmail.com"
FROM_EMAIL = my_credentials['user']
FROM_PWD = my_credentials['password']
SMTP_SERVER = "imap.gmail.com"
SMTP_PORT = 993

def read_email():

    From=[]
    Subject=[]
    Body=[]
    try:
        mail = imaplib.IMAP4_SSL(SMTP_SERVER)
        mail.login(FROM_EMAIL,FROM_PWD)
        mail.select('inbox')

        data = mail.search(None, 'ALL')
        mail_ids = data[1]
        id_list = mail_ids[0].split()
        first_email_id = int(id_list[0])
        latest_email_id = int(id_list[-1])

        # reading top 5 emails
        start = latest_email_id
        end = latest_email_id - 5

        for i in range(start,end, -1):
            data = mail.fetch(str(i), '(RFC822)' )
            for response_part in data:
                arr = response_part[0]
                if isinstance(arr, tuple):
                    msg = email.message_from_string(str(arr[1],'utf-8'))
                    email_subject = msg['subject']
                    email_from = msg['from']
                    # print('From : ' + email_from + '\n')
                    From.append(email_from)
                    # print('Subject : ' + email_subject + '\n')
                    Subject.append(email_subject)
                    if msg.is_multipart():
                        # If the email has multiple parts, iterate over them
                        for part in msg.walk():
                            # Check if the content type is text/plain
                            if part.get_content_type() == "text/plain":
                                # Get the email body and print it
                                email_body = part.get_payload(decode=True).decode()
                                # print('Message : ' + email_body + '\n')
                                Body.append(email_body)
                    else:
                        # If the email is not multipart, directly access the body
                        email_body = msg.get_payload(decode=True).decode()
                        # print('Message : ' + email_body + '\n')
                        Body.append(email_body)

    except Exception as e:
        traceback.print_exc()
        print(str(e))

    return From,Subject,Body

From,Subject,Body=read_email()


In [16]:
Subject_pattern = r'(?i)\b(?:meeting|conference|meet|talk|zoom meeting|zoom)\b'
ind =[]

for i in range(0,len(Subject)):

  matches = re.findall(Subject_pattern,Subject[i])
  if(len(matches)>=1):
    ind.append(i)

ind

[0, 3]

In [21]:

def extract_data_from_text(text):

      # Extracting the topic
      topic_match = re.search(r"Topic - (.+)", text)
      topic = topic_match.group(1).strip() if topic_match else None

      # Extracting the time
      time_match = re.search(r"Time - (.+)", text)
      time = time_match.group(1).strip() if time_match else None

      # Extracting the type
      type_match = re.search(r"Type - (.+)", text)
      type = type_match.group(1).strip() if type_match else None

      # Extracting the host
      host_match = re.search(r"Host - (.+)", text)
      host = host_match.group(1).strip() if host_match else None

      # Extracting the date
      date_match = re.search(r"Date - (.+)", text)
      date = date_match.group(1).strip() if date_match else None


      # extracted information
      extracted_res={'topic':'','time':'','type':'','host':'','date':''}
      extracted_res['topic']=topic
      extracted_res['time']=time
      extracted_res['type']=type
      extracted_res['host']=host
      extracted_res['date']=date

      return extracted_res

In [22]:
res = extract_data_from_text(Body[ind[0]])
res

{'topic': 'RPA test final',
 'time': '2:00pm',
 'type': None,
 'host': None,
 'date': '2023-07-13'}

In [23]:
def type_slowly(word):
    for character in word:
        pyautogui.write(character)
        time.sleep(0.1)

### Zoom meeting creating


In [24]:
pyautogui.press('win')  # Press the Windows key
time.sleep(2)
pyautogui.typewrite('zoom') 

time.sleep(2)
pyautogui.press('enter')  # Press Enter to open Zoom
time.sleep(3)  # Wait for Zoom to open

# writting data
topic=res["topic"]
host=res["host"]
start_time=res["time"]
end_time=''
button7location = pyautogui.locateOnScreen('schedule.png') # returns (left, top, width, height) of matching region
time.sleep(3)
pyautogui.click(pyautogui.moveTo(button7location))
time.sleep(6) 
type_slowly(topic)
pyautogui.press('tab')
time.sleep(1)
pyautogui.press('tab')
time.sleep(2)
pyautogui.press('tab')
time.sleep(1)
pyautogui.press('tab')
type_slowly(start_time)
# testing
pyautogui.press('tab')
time.sleep(1)
pyautogui.press('tab')
pyautogui.press('tab')
time.sleep(1)
pyautogui.press('tab')
pyautogui.press('tab')
time.sleep(1)
pyautogui.press('tab')
time.sleep(1)
pyautogui.press('enter')
# till here
time.sleep(3)
time.sleep(5)
button7location = pyautogui.locateOnScreen('copy_link.png') # returns (left, top, width, height) of matching region
time.sleep(2)
pyautogui.click(pyautogui.moveTo(button7location))
pyautogui.hotkey('alt', 'f4')
pyautogui.hotkey('alt', 'f4')
pyautogui.hotkey('ctrl', 'v')
link =clipboard.paste()
print(link)


RPA test final
sonupas2806@gmail.com


### Code for sending mail

In [59]:


# Create a MIME text object with the email content
message = MIMEText(link)

# Set the sender and recipient addresses
FROM = my_credentials["user"]
message["From"] = FROM
message["To"] = my_credentials["to"]
message["Subject"] = "Zoom meeting"

# Connect to the SMTP server
smtp_server = "smtp.gmail.com"
smtp_port = 587
smtp_password = my_credentials["password"]

with smtplib.SMTP(smtp_server, smtp_port) as server:
    # Start the secure connection if using TLS
    server.starttls()

    # Login to the SMTP server if authentication is required
    server.login(FROM, smtp_password)

    # Send the email
    server.send_message(message)

    # Close the connection
    print("message sent successfully")
    server.quit()


message sent successfully
